In [4]:
import boto3
import importlib
import json
import os
import pathlib
import sys

import awswrangler as wr

from botocore.exceptions import ClientError
from IPython.display import display

# Adding ../01_modules or ./01_modules to the system path so that we can load modules from 
# there as well
if '__file__' in globals():
    script_dir = pathlib.Path(__file__).parent.resolve()
else:
    script_dir = pathlib.Path().absolute()
modules_path_in_dev = os.path.abspath(os.path.join(script_dir, '..', '01_modules'))
modules_path_in_prod = os.path.abspath(os.path.join(script_dir, '01_modules'))
if os.path.exists(modules_path_in_dev):
    sys.path.append(modules_path_in_dev)
if os.path.exists(modules_path_in_prod):
    sys.path.append(modules_path_in_prod)


# # Jupyter only reads a local module the first time after 
# # kernel start. Re-running a cell with 
# # "from mymodulename import *" would not change
# # anything, even if the imported module has since changed.
# # As a workaround, we need to directly load the module, 
# # use importlib.reload to reload it and then import * 
import utils
_ = importlib.reload(utils)
import config
_ = importlib.reload(config) 

utils.py loaded: v0.2.12
config.py loaded: v0.1


In [5]:
q = utils.get_available_training_quotas()
q

{'ml.g6.xlarge': {'quota': 6, 'usage': 0, 'available': 6},
 'ml.g6.2xlarge': {'quota': 3, 'usage': 0, 'available': 3},
 'ml.g6.4xlarge': {'quota': 6, 'usage': 0, 'available': 6},
 'ml.g6.8xlarge': {'quota': 6, 'usage': 0, 'available': 6}}

In [6]:
sum([v['usage'] for k,v in q.items()])

0